# BBOX3 Baseline Optimizer

This notebook implements the bbox3 optimization approach for the Santa 2025 Christmas tree packing problem.

In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity
from shapely.strtree import STRtree
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar
from scipy.spatial import ConvexHull
import subprocess
import os
import time

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def make_tree_polygon(x, y, deg):
    """Create a tree polygon at position (x, y) with rotation deg."""
    coords = list(zip(TX, TY))
    poly = Polygon(coords)
    poly = affinity.rotate(poly, deg, origin=(0, 0))
    poly = affinity.translate(poly, x, y)
    return poly

def load_submission(path):
    """Load submission CSV and return dataframe."""
    df = pd.read_csv(path)
    df['xf'] = df['x'].str[1:].astype(float)
    df['yf'] = df['y'].str[1:].astype(float)
    df['degf'] = df['deg'].str[1:].astype(float)
    df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))
    df['t'] = df['id'].apply(lambda x: int(x.split('_')[1]))
    return df

def get_config_trees(df, n):
    """Get trees for configuration n."""
    config = df[df['n'] == n].sort_values('t')
    trees = []
    for _, row in config.iterrows():
        trees.append({
            'x': row['xf'],
            'y': row['yf'],
            'deg': row['degf'],
            'polygon': make_tree_polygon(row['xf'], row['yf'], row['degf'])
        })
    return trees

def get_bounding_box_side(trees):
    """Calculate the side length of the square bounding box."""
    if not trees:
        return 0
    all_coords = []
    for tree in trees:
        coords = list(tree['polygon'].exterior.coords)
        all_coords.extend(coords)
    xs = [c[0] for c in all_coords]
    ys = [c[1] for c in all_coords]
    return max(max(xs) - min(xs), max(ys) - min(ys))

def calculate_score(df):
    """Calculate total score for submission."""
    total_score = 0
    for n in range(1, 201):
        trees = get_config_trees(df, n)
        if trees:
            side = get_bounding_box_side(trees)
            score = (side ** 2) / n
            total_score += score
    return total_score

def has_overlap(trees):
    """Check if any trees overlap."""
    if len(trees) < 2:
        return False
    polygons = [t['polygon'] for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx != i and poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

print("Functions loaded successfully!")
print(f"Tree polygon vertices: {len(TX)}")
print(f"Tree width: {max(TX) - min(TX):.3f}")
print(f"Tree height: {max(TY) - min(TY):.3f}")

In [ ]:
# Load and score the sample submission
sample_path = '/home/data/sample_submission.csv'
df_sample = load_submission(sample_path)

print("Calculating sample submission score...")
sample_score = calculate_score(df_sample)
print(f"Sample submission score: {sample_score:.6f}")

# Check for overlaps in sample
print("\nChecking for overlaps in sample submission...")
overlap_count = 0
for n in range(1, 201):
    trees = get_config_trees(df_sample, n)
    if has_overlap(trees):
        overlap_count += 1
        if overlap_count <= 5:
            print(f"  Overlap found in N={n}")
print(f"Total configurations with overlaps: {overlap_count}")

In [ ]:
# Analyze score breakdown by configuration size
scores_by_n = []
for n in range(1, 201):
    trees = get_config_trees(df_sample, n)
    if trees:
        side = get_bounding_box_side(trees)
        score = (side ** 2) / n
        scores_by_n.append({'n': n, 'side': side, 'score': score})

scores_df = pd.DataFrame(scores_by_n)
print("Score breakdown:")
print(f"  N=1-10 total: {scores_df[scores_df['n'] <= 10]['score'].sum():.4f}")
print(f"  N=11-50 total: {scores_df[(scores_df['n'] > 10) & (scores_df['n'] <= 50)]['score'].sum():.4f}")
print(f"  N=51-100 total: {scores_df[(scores_df['n'] > 50) & (scores_df['n'] <= 100)]['score'].sum():.4f}")
print(f"  N=101-200 total: {scores_df[scores_df['n'] > 100]['score'].sum():.4f}")

# Plot score vs n
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(scores_df['n'], scores_df['score'])
plt.xlabel('N (number of trees)')
plt.ylabel('Score (side²/n)')
plt.title('Score by Configuration Size')

plt.subplot(1, 2, 2)
plt.plot(scores_df['n'], scores_df['side'])
plt.xlabel('N (number of trees)')
plt.ylabel('Bounding Box Side')
plt.title('Bounding Box Side by Configuration Size')

plt.tight_layout()
plt.savefig('score_analysis.png', dpi=100)
plt.show()

In [ ]:
# Rotation optimization function (fix_direction)
def calculate_bbox_side_at_angle(angle, hull_points):
    """Calculate bounding box side after rotating by angle degrees."""
    rad = np.radians(angle)
    cos_a, sin_a = np.cos(rad), np.sin(rad)
    rotated = hull_points @ np.array([[cos_a, -sin_a], [sin_a, cos_a]]).T
    width = rotated[:, 0].max() - rotated[:, 0].min()
    height = rotated[:, 1].max() - rotated[:, 1].min()
    return max(width, height)

def optimize_rotation(trees):
    """Find optimal rotation angle to minimize bounding box."""
    if not trees:
        return 0, trees
    
    # Get all polygon vertices
    all_points = []
    for tree in trees:
        coords = list(tree['polygon'].exterior.coords)
        all_points.extend(coords)
    points_np = np.array(all_points)
    
    # Get convex hull
    try:
        hull = ConvexHull(points_np)
        hull_points = points_np[hull.vertices]
    except:
        hull_points = points_np
    
    # Find optimal rotation angle
    result = minimize_scalar(
        lambda a: calculate_bbox_side_at_angle(a, hull_points),
        bounds=(0.001, 89.999),
        method='bounded'
    )
    
    best_angle = result.x
    
    # Apply rotation to all trees
    rotated_trees = []
    for tree in trees:
        # Rotate position around origin
        rad = np.radians(best_angle)
        cos_a, sin_a = np.cos(rad), np.sin(rad)
        new_x = tree['x'] * cos_a - tree['y'] * sin_a
        new_y = tree['x'] * sin_a + tree['y'] * cos_a
        new_deg = tree['deg'] + best_angle
        
        rotated_trees.append({
            'x': new_x,
            'y': new_y,
            'deg': new_deg,
            'polygon': make_tree_polygon(new_x, new_y, new_deg)
        })
    
    return best_angle, rotated_trees

# Test rotation optimization on a few configurations
print("Testing rotation optimization...")
for n in [10, 50, 100, 200]:
    trees = get_config_trees(df_sample, n)
    orig_side = get_bounding_box_side(trees)
    angle, rotated = optimize_rotation(trees)
    new_side = get_bounding_box_side(rotated)
    improvement = (orig_side - new_side) / orig_side * 100
    print(f"N={n}: {orig_side:.4f} -> {new_side:.4f} (angle={angle:.2f}°, improvement={improvement:.2f}%)")


In [ ]:
# Apply rotation optimization to all configurations
print("Applying rotation optimization to all configurations...")

optimized_data = []
for n in range(1, 201):
    trees = get_config_trees(df_sample, n)
    if trees:
        angle, rotated = optimize_rotation(trees)
        for t_idx, tree in enumerate(rotated):
            optimized_data.append({
                'id': f'{n:03d}_{t_idx}',
                'x': f's{tree["x"]}',
                'y': f's{tree["y"]}',
                'deg': f's{tree["deg"]}'
            })
    if n % 50 == 0:
        print(f"  Processed N=1 to {n}")

df_optimized = pd.DataFrame(optimized_data)
df_optimized.to_csv('optimized_submission.csv', index=False)
print(f"\nSaved optimized submission to optimized_submission.csv")

# Calculate new score
df_opt_loaded = load_submission('optimized_submission.csv')
opt_score = calculate_score(df_opt_loaded)
print(f"\nOriginal score: {sample_score:.6f}")
print(f"Optimized score: {opt_score:.6f}")
print(f"Improvement: {sample_score - opt_score:.6f} ({(sample_score - opt_score)/sample_score*100:.2f}%)")


In [ ]:
# Validate optimized submission for overlaps
print("Validating optimized submission...")
overlap_configs = []
for n in range(1, 201):
    trees = get_config_trees(df_opt_loaded, n)
    if has_overlap(trees):
        overlap_configs.append(n)

if overlap_configs:
    print(f"WARNING: Overlaps found in {len(overlap_configs)} configurations: {overlap_configs[:10]}...")
else:
    print("SUCCESS: No overlaps found in optimized submission!")

# Copy to submission folder
import shutil
shutil.copy('optimized_submission.csv', '/home/submission/submission.csv')
print("\nCopied to /home/submission/submission.csv")

In [ ]:
# Final score summary
print("="*50)
print("FINAL RESULTS")
print("="*50)
print(f"Sample submission score: {sample_score:.6f}")
print(f"Optimized submission score: {opt_score:.6f}")
print(f"Target score to beat: 68.922808")
print(f"Current gap to target: {opt_score - 68.922808:.6f}")
print("="*50)